***
____
![revit](https://i.ibb.co/bQ3dB8C/curso-revit.png)

***
***


# Clase 07
## Selección de variables

In [1]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10, 10]

Para garantizar la reproducibilidad (esto es, que al ejecutar este notebook otra vez los resultados sean idénticos) vamos a fijar la semilla que usa scikit learn para hacer particiones aleatorias. Para esto tenemos que fijar la semilla (seed) cada vez que llamemos a la aplicacion.

In [2]:
 np.random.seed(42)

# Selección de variables

En esta sección vamos a ver las distintas técnicas de selección de variables, que son las técnicas que nos permiten filtrar las variables que tenemos y elegir aquellas que realmente están ayudando al modelo.

¿Por qué querriamos eliminar variables? Generalmente, cuanta más información tenemos sobre cada observación mejor será la calidad de nuestras predicciones (cuanto más sepamos, más facil será diferenciar unos casos de otros). No obstante, esto no tiene por que ser así en todas las situaciones, hay veces que ciertas variables pueden ser contraproducentes y hacer que nuestros modelos funcionen peor, por ejemplo debido a errores de medición.

Seleccionar variables tiene varias ventajas. En primer lugar, como hemos dicho, puede mejorar las puntuaciones de los modelos (eliminando parte del ruido en los datos), por otra parte, reducir el número de variables independientes reduce la complejidad del modelo, reduciendo así el efecto de sobreajuste del modelo. Además, tener menos variables independientes y modelos más simples significa que nuestros modelos son más rápidos de entrenar, y podemos entrenarlos con más datos en las mismas máquinas.

## Cargamos los datos

Cargaremos los datos del data `Caso mora en tarjetas de crédito`

In [8]:
datos = pd.read_csv("../Dataset/Consumo_alcohol.csv").drop(["ID"], axis = 1)

In [9]:
datos.shape

(1044, 33)

In [10]:
datos.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,health,absences,G1,G2,G3,curso,alcohol
0,GP,F,18,U,GT3,A,higher education,higher education,at_home,teacher,...,4,3,4,3,6,5,6,6,mat,No
1,GP,F,17,U,GT3,T,primary education (4th grade),primary education (4th grade),at_home,other,...,5,3,3,3,4,5,5,6,mat,No
2,GP,F,15,U,LE3,T,primary education (4th grade),primary education (4th grade),at_home,other,...,4,3,2,3,10,7,8,10,mat,No
3,GP,F,15,U,GT3,T,higher education,5th to 9th grade,health,services,...,3,2,2,5,2,15,14,15,mat,No
4,GP,F,16,U,GT3,T,secondary education,secondary education,other,other,...,4,3,2,5,4,6,10,10,mat,No


In [11]:
datos.sample(1).T

,971
school,MS
sex,M
age,18
address,R
famsize,GT3
Pstatus,T
Medu,secondary education
Fedu,5th to 9th grade
Mjob,other
Fjob,other


In [19]:


#import datetime as dt

#datos.VAR04 = pd.to_datetime(datos.VAR04)
#datos["anio_ingreso"] = datos.VAR04.dt.year
#datos["tiemp_trab"] = 2019 - datos.anio_ingreso

#datos = datos.drop(["VAR04", "anio_ingreso"], axis = 1)

In [20]:
datos.dtypes

school        category
sex             object
age              int64
address         object
famsize         object
Pstatus         object
Medu            object
Fedu            object
Mjob            object
Fjob            object
reason          object
guardian        object
traveltime      object
studytime       object
failures         int64
schoolsup       object
famsup          object
paid            object
activities      object
nursery         object
higher          object
internet        object
romantic        object
famrel           int64
freetime         int64
goout            int64
health           int64
absences         int64
G1               int64
G2               int64
G3               int64
curso           object
alcohol         object
dtype: object

## Procesado de datos

In [21]:


# Separamos los datos numéricos y categóricos
datos_numericos = datos.select_dtypes(include=['float64', "int64"])
datos_categoricos = datos.select_dtypes(exclude=['float64', "int64"])

# Para los missing numéricos los imputamos con la media
for col in datos_numericos.columns:
    datos_numericos[col].fillna(datos_numericos[col].mean(), inplace=True)

# Para los categoricos creamos dummies
datos_categoricos_codificados = pd.get_dummies(datos_categoricos)
df_final = pd.concat([datos_numericos, datos_categoricos_codificados], axis=1)

In [22]:
df_final.shape

(1044, 75)

In [26]:
df_final.head()

,age,failures,famrel,freetime,goout,health,absences,G1,G2,G3,...,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes,curso_mat,curso_por,alcohol_No,alcohol_Si
0,18,0,4,3,4,3,6,5,6,6,...,0,1,1,0,1,0,1,0,1,0
1,17,0,5,3,3,3,4,5,5,6,...,0,1,0,1,1,0,1,0,1,0
2,15,3,4,3,2,3,10,7,8,10,...,0,1,0,1,1,0,1,0,1,0
3,15,0,3,2,2,5,2,15,14,15,...,0,1,0,1,0,1,1,0,1,0
4,16,0,4,3,2,5,4,6,10,10,...,0,1,1,0,1,0,1,0,1,0


Ya tenemos un dataset preparado para poder entrenar modelos. Este dataset tiene una complejidad dimensional alta ( 986 variables independientes), por lo que vamos a usar técnicas de selección de variables para reducirla.

Antes que nada vamos a ver que errores obtenemos con diversos modelos entrenando con el dataset con todas las variables.

In [24]:
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

objetivo = "alcohol"
X=df_final.drop(objetivo, axis=1)
y=df_final[objetivo]

KeyError: "['alcohol'] not found in axis"

Vamos a usar la función `cross_validate` que es una versión más flexible que `cross_val_score`. Evaluaremos usando el área bajo la curva ROC

In [12]:
clasificador = cross_validate(LogisticRegression(), X, y, scoring="roc_auc", n_jobs=-1, cv=10)

Dicha función devuelve no solo las puntuaciones en los datos de testing, sino también en los datos de entrenamiento. Además, nos devuelve el tiempo que tarda dicho estimador en ajustar el modelo y en hacer predicciones con el mismo.

In [13]:
pd.DataFrame(clasificador)

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


,fit_time,score_time,test_score,train_score
0,0.349067,0.006980,0.637010,0.728067
1,0.175531,0.005985,0.644933,0.709180
2,0.263299,0.028922,0.662743,0.707796
3,0.198471,0.007976,0.735238,0.689666
4,0.210437,0.002992,0.729486,0.683121
5,0.307179,0.002993,0.786190,0.719119
6,0.331114,0.002991,0.774629,0.722197
7,0.197472,0.026928,0.734095,0.692050
8,0.255308,0.002993,0.823924,0.722961
9,0.243349,0.002991,0.769276,0.736491


In [14]:
def evaluar_modelo(estimador, X, y):
    resultados_estimador = cross_validate(estimador, X, y,
                     scoring="roc_auc", n_jobs=-1, cv=10, return_train_score=True)
    return resultados_estimador

In [15]:
resultados = {}

def ver_resultados():
    resultados_df  = pd.DataFrame(resultados).T
    resultados_cols = resultados_df.columns
    for col in resultados_df:
        resultados_df[col] = resultados_df[col].apply(np.mean)
        resultados_df[col+"_idx"] = resultados_df[col] / resultados_df[col].min()
    return resultados_df

In [16]:
resultados["reg_logis_sin_seleccion"] = evaluar_modelo(LogisticRegression(), X, y)
resultados["knn_sin_seleccion"] = evaluar_modelo(KNeighborsClassifier(n_neighbors=500, weights="distance"), X, y)
resultados["rf_sin_seleccion"] = evaluar_modelo(RandomForestClassifier(n_estimators=100), X, y)

In [17]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logis_sin_seleccion,0.318947,0.006383,0.729752,0.711065,9.993962,1.000000,1.077675,1.000000
knn_sin_seleccion,0.031914,0.317251,0.677154,1.000000,1.000000,49.705242,1.000000,1.406342
rf_sin_seleccion,1.464084,0.040592,0.762545,1.000000,45.875918,6.359668,1.126102,1.406342


Hay 3 tipos generales de estrategias de selección de variables:

# Métodos de filtrado

Los métodos de filtrado usan métodos estadísticos para seleccionar las variables que proporcionan la mayor cantidad de información. Estos métodos se aplican de forma previa a entrenar el modelo (preprocesado), y **son completamente independientes de la elección del estimador**. Generalmente funcionan definiendo una función de evaluación $S(xk_i, y_k)$, evaluando cada variable independiente para cada observación respecto a la variable objetivo de dicha observación, y eligiendo aquellas `K` variables que mejor funcionan.



Scikit-learn tiene las siguientes funciones de evaluación:

- Para regresión: `f_regression, mutual_info_regression`
- Para clasificación: `chi2, f_classif, mutual_info_classif`

f_regression y f_classif devuelven [estadísticos F](https://es.wikipedia.org/wiki/Estad%C3%ADstico_F) (F-values), entrenando un modelo lineal entre las variables independientes y la objetivo en el caso de regresión, y un test ANOVA en el caso de clasificación.

`mutual_info_regression` y `mutual_info_classif` computan el coeficiente de información mutua (MIC) entre las variables independientes y la variable objetivo.

El coeficiente de información mutua nos da una medida de la dependencia entre las variables. El MIC entre dos variables es 0 si no hay relación entre las mismas, y aumenta conforme más relación tienen.

El MIC se define como:

$$I(X;Y) = \sum_{x,y} P_{XY}(x,y) \log
{P_{XY}(x,y) \over P_X(x) P_Y(y)}$$

con $P_{XY}$ siendo la probabilidad conjunta de X e Y


El evaluador `chi2` calcula el estadístico chi cuadrado y lo convierte a estadísticos F, con el estadístico chi cuadrado definido como:
  $$\chi^2(x,y) = \sum {\frac{(x-y)^2}{y}}$$


In [18]:
from sklearn.feature_selection import SelectKBest, f_classif

In [19]:
selector_kbest10 = SelectKBest(f_classif, k=20)
X_kbest10 = selector_kbest10.fit_transform(X, y)

In [20]:
X_kbest10.shape

(5000, 20)

La funcion `get_support` nos devuelve un vector booleano (True/False), aquellos elementos con True son las columnas que se han seleccionado

In [21]:
columnas_seleccion_kbest10 = X.loc[:,selector_kbest10.get_support()].columns
columnas_seleccion_kbest10

Index(['VAR01', 'VAR03', 'VAR05', 'VAR06', 'VAR07', 'VAR08', 'VAR10',
       'tiemp_trab', 'VAR02_FAMILIAR', 'VAR02_PROPIA', 'VAR12_Arequipa',
       'VAR12_Lima', 'VAR12_Loreto', 'VAR12_Piura', 'VAR13_0', 'VAR13_1',
       'VAR13_2', 'VAR14_EDUCACION BASICA', 'VAR14_PROFESIONAL',
       'VAR14_TECNICO'],
      dtype='object')

El parámetro `scores_` del selector nos devuelve los resultados de la función de evaluación

In [22]:
selector_kbest10.scores_[:10]

array([3.05976615e+01, 6.84267123e+01, 3.81019993e+02, 5.33338844e+01,
       1.31605728e+02, 1.69440570e+02, 4.69990311e-03, 2.67668523e+02,
       8.34999653e+00, 1.00818867e+02])

In [23]:
evaluacion_kbest10 = pd.DataFrame({"variable":X.columns, 
                                   "Score":selector_kbest10.scores_, 
                                   "Seleccionado":selector_kbest10.get_support()})

In [24]:
evaluacion_kbest10

,variable,Score,Seleccionado
0,VAR01,30.597662,True
1,VAR03,68.426712,True
2,VAR05,381.019993,True
3,VAR06,53.333884,True
4,VAR07,131.605728,True
5,VAR08,169.440570,True
6,VAR09,0.004700,False
7,VAR10,267.668523,True
8,VAR11,8.349997,False
9,tiemp_trab,100.818867,True


Esto nos permite ver cual es la puntuación que le da el evaluador `f_regression` a cada variable independiente

In [25]:
resultados["reg_logis_kbest_10"] = evaluar_modelo(LogisticRegression(), X_kbest10, y)
resultados["knn_kbest_10"] = evaluar_modelo(KNeighborsClassifier(n_neighbors=500, weights="distance"), X_kbest10, y)
resultados["rf_kbest_10"] = evaluar_modelo(RandomForestClassifier(n_estimators=100), X_kbest10, y)

In [26]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logis_sin_seleccion,0.318947,0.006383,0.729752,0.711065,20.634473,2.463110,1.099618,1.009406
knn_sin_seleccion,0.031914,0.317251,0.677154,1.000000,2.064694,122.429463,1.020361,1.419570
rf_sin_seleccion,1.464084,0.040592,0.762545,1.000000,94.719730,15.664560,1.149030,1.419570
reg_logis_kbest_10,0.140525,0.002591,0.736200,0.704439,9.091352,1.000000,1.109333,1.000000
knn_kbest_10,0.015457,0.152692,0.663642,1.000000,1.000000,58.924830,1.000000,1.419570
rf_kbest_10,0.961328,0.030020,0.742344,1.000000,62.193638,11.584854,1.118591,1.419570


Vemos que al reducir las dimensiones a solo 10, los modelos lineales y RF funcionan bastante peor, aunque vemos que por otra parte las diferencias entre la evaluación de entrenamiento y test se han reducido en todos los casos. Esto es así por que al tener menos variables, hay mucha menos flexibilidad para sobreajustar (es decir, "memorizar" los datos de entrenamiento). SVM sin embargo funciona más o menos igual (de mal), pero los tiempos de entrenamiento se reducen considerablemente (de 3.9 segundos a 0.7).

Probamos ahora para 50 variables

In [27]:
selector_kbest50 = SelectKBest(f_classif, k=30)
X_kbest50 = selector_kbest50.fit_transform(X, y)

In [28]:
resultados["reg_logis_kbest_50"] = evaluar_modelo(LogisticRegression(), X_kbest50, y)
resultados["knn_kbest_50"] = evaluar_modelo(KNeighborsClassifier(n_neighbors=500, weights="distance"), X_kbest50, y)
resultados["rf_kbest_50"] = evaluar_modelo(RandomForestClassifier(n_estimators=100), X_kbest50, y)

In [29]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logis_sin_seleccion,0.318947,0.006383,0.729752,0.711065,20.634473,3.764914,1.099618,1.009406
knn_sin_seleccion,0.031914,0.317251,0.677154,1.000000,2.064694,187.135980,1.020361,1.419570
rf_sin_seleccion,1.464084,0.040592,0.762545,1.000000,94.719730,23.943605,1.149030,1.419570
reg_logis_kbest_10,0.140525,0.002591,0.736200,0.704439,9.091352,1.528521,1.109333,1.000000
knn_kbest_10,0.015457,0.152692,0.663642,1.000000,1.000000,90.067828,1.000000,1.419570
rf_kbest_10,0.961328,0.030020,0.742344,1.000000,62.193638,17.707690,1.118591,1.419570
reg_logis_kbest_50,0.162266,0.001695,0.752272,0.722616,10.497907,1.000000,1.133552,1.025804
knn_kbest_50,0.019348,0.162066,0.663901,1.000000,1.251733,95.597545,1.000390,1.419570
rf_kbest_50,1.044406,0.027128,0.747224,1.000000,67.568461,16.001716,1.125944,1.419570


Vemos que para 50 variables, los tres estimadores funcionan de forma similar al modelo entrenado con las 209 variables iniciales, sin embargo los tiempos de entrenamiento de los mismos se reducen considerablemente.

De forma similar a `SelectKBest`, donde le decimos al transformador el número de variables que queremos mantener, podemos usar [`SelectPercentile`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile) donde especificamos el porcentaje de variables sobre el total.

Por ejemplo, si solo quisieramos conservar el 10% de las variables (sin importar que tengamos 200 o 20000), podemos hacer lo siguiente:


In [30]:
from sklearn.feature_selection import SelectPercentile

selector_pct10 = SelectPercentile(f_classif, percentile=10)
X_pct10 = selector_pct10.fit_transform(X, y)

In [31]:
X_pct10.shape

(5000, 5)

El transformador nos selecciona 98 variables (el 10 por ciento de 981 redondeado)

# Métodos envolventes (wrapper methods)

Los métodos envolventes *(wrapper methods)* funcionan de forma similar a los métodos de ranking. Sin embargo, en lugar de usar una función estadística independiente del modelo para evaluar las variables, estos métodos usan la función de evaluación o el performance de los modelos como input para decidir que variables elegir (es decir, "envuelven" el funcionamiento del estimador). Ésto significa que los métodos de filtrado se pueden aplicar independientemente de la elección del modelo, ya que consideran los modelos como una caja negra que produce evaluaciones, aunque claro, diferentes modelos producirán diferentes selecciones de variables.

`Scikit-learn` implementa un metodo envolvente llamado **Recursive Feature Elimination [(RFE)](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination)**, o Eliminación Recursiva de Variables. RFE funciona seleccionando todas las variables, entrenando el modelo, usando los coeficientes `coef_` o la importancia de las variables `feature_importances_` en función del estimador, y eliminando n variables. Este proceso se repite hasta que se alcanza el número de variables deseado.

In [32]:
from sklearn.feature_selection import RFE
estimador_selector = RandomForestClassifier()
selector_rfe10_rf = RFE(estimador_selector, n_features_to_select=20)
X_rfe10_rf = selector_rfe10_rf.fit_transform(X, y)

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

In [33]:
X_rfe10_rf.shape

(5000, 20)

In [34]:
evaluacion_rfe10_rf = sorted(
    filter(lambda c: c[2], 
        zip(
            X.columns,
            selector_rfe10_rf.ranking_,
            selector_rfe10_rf.get_support()
        )
    ), key=lambda c: c[1],reverse=True
)

In [35]:
evaluacion_rfe10_rf

[('VAR01', 1, True),
 ('VAR03', 1, True),
 ('VAR05', 1, True),
 ('VAR06', 1, True),
 ('VAR07', 1, True),
 ('VAR08', 1, True),
 ('VAR09', 1, True),
 ('VAR10', 1, True),
 ('VAR11', 1, True),
 ('tiemp_trab', 1, True),
 ('VAR02_FAMILIAR', 1, True),
 ('VAR02_PROPIA', 1, True),
 ('VAR12_Arequipa', 1, True),
 ('VAR12_Callao', 1, True),
 ('VAR12_La Libertad', 1, True),
 ('VAR12_Lima', 1, True),
 ('VAR13_0', 1, True),
 ('VAR13_1', 1, True),
 ('VAR14_PROFESIONAL', 1, True),
 ('VAR14_TECNICO', 1, True)]

In [36]:
resultados["reg_lineal_rfe10_rf"] = evaluar_modelo(LogisticRegression(), X_rfe10_rf, y)
resultados["rf_rfe10_rf"] = evaluar_modelo(RandomForestClassifier(), X_rfe10_rf, y)
resultados["svr_rfe10_rf"] = evaluar_modelo(SVC(), X_rfe10_rf, y)

In [37]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logis_sin_seleccion,0.318947,0.006383,0.729752,0.711065,20.634473,3.764914,1.444684,1.010185
knn_sin_seleccion,0.031914,0.317251,0.677154,1.000000,2.064694,187.135980,1.340556,1.420666
rf_sin_seleccion,1.464084,0.040592,0.762545,1.000000,94.719730,23.943605,1.509602,1.420666
reg_logis_kbest_10,0.140525,0.002591,0.736200,0.704439,9.091352,1.528521,1.457448,1.000772
knn_kbest_10,0.015457,0.152692,0.663642,1.000000,1.000000,90.067828,1.313805,1.420666
rf_kbest_10,0.961328,0.030020,0.742344,1.000000,62.193638,17.707690,1.469611,1.420666
reg_logis_kbest_50,0.162266,0.001695,0.752272,0.722616,10.497907,1.000000,1.489266,1.026596
knn_kbest_50,0.019348,0.162066,0.663901,1.000000,1.251733,95.597545,1.314318,1.420666
rf_kbest_50,1.044406,0.027128,0.747224,1.000000,67.568461,16.001716,1.479272,1.420666
reg_lineal_rfe10_rf,0.161368,0.003590,0.728962,0.703895,10.439778,2.117824,1.443119,1.000000


Si usamos otro estimador para evaluar veremos que las variables elegidas pueden ser completamente distintas. Los estimadores que se pueden usar tienen que implementar el metodo `coef_` o el metodo `feature_importance` (es decir, tienen que tener una manera de ordenar variables en función de su importancia). Por ejemplo, no podemos usar SVMs.

In [38]:
estimador_selector = LogisticRegression()
selector_rfe10_lineal = RFE(estimador_selector, n_features_to_select=20)
X_rfe10_lineal = selector_rfe10_lineal.fit_transform(X, y)

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

In [39]:
evaluacion_rfe10_lineal = sorted(
    filter(lambda c: c[2], 
        zip(
            X.columns,
            selector_rfe10_lineal.ranking_,
            selector_rfe10_lineal.get_support()
        )
    ), key=lambda c: c[1],reverse=True
)
evaluacion_rfe10_lineal

[('VAR02_ALQUILADA', 1, True),
 ('VAR02_FAMILIAR', 1, True),
 ('VAR02_OTRAS', 1, True),
 ('VAR12_Ancash', 1, True),
 ('VAR12_Arequipa', 1, True),
 ('VAR12_Huancavelica', 1, True),
 ('VAR12_Ica', 1, True),
 ('VAR12_La Libertad', 1, True),
 ('VAR12_Loreto', 1, True),
 ('VAR12_Moquegua', 1, True),
 ('VAR12_Pasco', 1, True),
 ('VAR12_Piura', 1, True),
 ('VAR12_San Martin', 1, True),
 ('VAR12_Tacna', 1, True),
 ('VAR12_Tumbes', 1, True),
 ('VAR12_Ucayali', 1, True),
 ('VAR13_0', 1, True),
 ('VAR13_3', 1, True),
 ('VAR14_EDUCACION BASICA', 1, True),
 ('VAR14_TECNICO', 1, True)]

In [40]:
set(evaluacion_rfe10_rf).intersection(set(evaluacion_rfe10_lineal))

{('VAR02_FAMILIAR', 1, True),
 ('VAR12_Arequipa', 1, True),
 ('VAR12_La Libertad', 1, True),
 ('VAR13_0', 1, True),
 ('VAR14_TECNICO', 1, True)}

In [41]:
resultados["reg_lineal_rfe10_lineal"] = evaluar_modelo(LogisticRegression(), X_rfe10_lineal, y)
resultados["rf_rfe10_lineal"] = evaluar_modelo(RandomForestClassifier(), X_rfe10_lineal, y)
resultados["svr_rfe10_lineal"] = evaluar_modelo(SVC(), X_rfe10_lineal, y)

In [42]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logis_sin_seleccion,0.318947,0.006383,0.729752,0.711065,20.634473,3.764914,1.444684,1.179611
knn_sin_seleccion,0.031914,0.317251,0.677154,1.000000,2.064694,187.135980,1.340556,1.658937
rf_sin_seleccion,1.464084,0.040592,0.762545,1.000000,94.719730,23.943605,1.509602,1.658937
reg_logis_kbest_10,0.140525,0.002591,0.736200,0.704439,9.091352,1.528521,1.457448,1.168619
knn_kbest_10,0.015457,0.152692,0.663642,1.000000,1.000000,90.067828,1.313805,1.658937
rf_kbest_10,0.961328,0.030020,0.742344,1.000000,62.193638,17.707690,1.469611,1.658937
reg_logis_kbest_50,0.162266,0.001695,0.752272,0.722616,10.497907,1.000000,1.489266,1.198774
knn_kbest_50,0.019348,0.162066,0.663901,1.000000,1.251733,95.597545,1.314318,1.658937
rf_kbest_50,1.044406,0.027128,0.747224,1.000000,67.568461,16.001716,1.479272,1.658937
reg_lineal_rfe10_rf,0.161368,0.003590,0.728962,0.703895,10.439778,2.117824,1.443119,1.167718


Vemos que el estimador RF entrenado con solo 10 variables seleccionadas por `RFE` basado en RF es el modelo que mejor funciona, y tarda 8 veces menos en entrenar.

scikit-learn también tiene la implementación [RFECV](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html#sklearn.feature_selection.RFECV) que implementa el mismo algoritmo pero hace validación cruzada y entrena en múltiples particiones del dataset para calcular automáticamente el conjunto de variables que funciona mejor (a costa de un mayor tiempo de procesamiento).

In [43]:
from sklearn.feature_selection import RFECV

estimador_selector = RandomForestClassifier()
selector_rfecv = RFECV(estimador_selector)
X_rfecv = selector_rfecv.fit_transform(X, y)

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-p

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [48]:
X_rfecv.shape

(5000, 11)

En este caso `RFECV` ha elegido 188 variables como aquellas que proporcionan el mejor funcionamiento al estimador RF utilizado.

In [45]:
resultados["reg_lineal_rfecv_rf"] = evaluar_modelo(LogisticRegression(), X_rfecv, y)
resultados["rf_rfecv_rf"] = evaluar_modelo(RandomForestClassifier(), X_rfecv, y)
resultados["svr_rfecv_rf"] = evaluar_modelo(SVC(), X_rfecv, y)

In [46]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logis_sin_seleccion,0.318947,0.006383,0.729752,0.711065,20.634473,3.999462,1.453321,1.179611
knn_sin_seleccion,0.031914,0.317251,0.677154,1.000000,2.064694,198.794236,1.348570,1.658937
rf_sin_seleccion,1.464084,0.040592,0.762545,1.000000,94.719730,25.435252,1.518627,1.658937
reg_logis_kbest_10,0.140525,0.002591,0.736200,0.704439,9.091352,1.623745,1.466161,1.168619
knn_kbest_10,0.015457,0.152692,0.663642,1.000000,1.000000,95.678902,1.321660,1.658937
rf_kbest_10,0.961328,0.030020,0.742344,1.000000,62.193638,18.810849,1.478397,1.658937
reg_logis_kbest_50,0.162266,0.001695,0.752272,0.722616,10.497907,1.062298,1.498170,1.198774
knn_kbest_50,0.019348,0.162066,0.663901,1.000000,1.251733,101.553110,1.322176,1.658937
rf_kbest_50,1.044406,0.027128,0.747224,1.000000,67.568461,16.998596,1.488115,1.658937
reg_lineal_rfe10_rf,0.161368,0.003590,0.728962,0.703895,10.439778,2.249761,1.451746,1.167718


Vemos que el método que funciona mejor es el de RFECV usando RF como estimador. Este método funciona mejor que  el segundo mejor modelo, un Random Forest entrenado con todo el dataset, pero se entrena en menos tiempo (0.81 segundos versus  0.99 segundos)

# Metodos embebidos (Embedded Methods)

Éstos métodos son metodos de selección de variables que están integrados dentro del proceso de entrenamiento de modelos, y son específicos para cada modelo.

El ejemplo clásico de método embebidos de selección de variables son los procesos de regularización en regresión lineal. Por ejemplo el método L1 tiende a convertir los coeficientes de las variables que no funcionan a 0, por lo tanto eliminandolas.